## Imports

In [ ]:
import pandas as pd
import duckdb

## Initial Data Loading

In [8]:
locations = pd.read_csv('locations.csv')

display(locations)

,zip_code,location
0,10506,King's Landing
1,80976,Dorne
2,78956,Braavos
3,67305,Harrenhal
4,25089,Winterfell
5,48732,Karhold
6,35786,Tarth
7,93612,Dragonstone


In [9]:
jobs = pd.read_json('jobs.json')

display(jobs)

,updated_at,id,state,zip,price,company_id,posted_at
0,2021-11-05 22:35:00,94,expired,10506,152.60,0,2021-09-11 22:38:00
1,2022-06-29 12:31:00,19,posted,35786,416.87,1,2022-06-29 12:31:00
2,2021-01-06 16:47:00,20,posted,80976,351.77,6,2021-01-06 16:47:00
3,2021-07-16 07:52:00,70,expired,35786,442.07,4,2021-05-27 07:53:00
4,2022-11-16 06:53:00,54,cancelled,80976,441.55,3,2022-09-27 07:14:00
...,...,...,...,...,...,...,...
171,2022-10-03 08:19:00,73,expired,10506,77.28,2,2022-07-28 07:56:00
172,2021-07-15 20:07:00,60,posted,25089,63.72,3,2021-07-15 20:07:00
173,2022-03-29 05:39:00,38,posted,10506,131.99,6,2022-03-29 05:39:00
174,2021-09-28 18:44:00,9,posted,80976,499.23,4,2021-09-28 18:44:00


In [ ]:
companies = pd.read_json('companies.json')

# Convert Establishment Date from unix timestamp to date
companies["Establishment Date"] = pd.to_datetime(companies['Establishment Date'], unit='ms')

display(companies)

,Company ID,Company Name,Establishment Date,Number of Employees
0,0,Acme Inc.,2019-12-31,404
1,1,Best Corp.,2021-03-21,229
2,2,Bright Future Enterprises,2020-11-14,222
3,3,Delta Inc.,2020-11-06,662
4,4,Echo Enterprises,2022-05-05,255
5,5,Fast Track Inc.,2020-03-08,374
6,6,Global Enterprises,2022-02-03,896
7,7,High Hopes Inc.,2020-03-17,812
8,8,Infinite Solutions,2021-03-12,436
9,9,Jumpstart Corp.,2020-05-08,872


## Fact & Dimension Tables

In [ ]:
fact_job_state = duckdb.query("""

    WITH next_updated_at AS (
        SELECT
            j.id as job_id
            ,j.company_id
            ,j.zip as zip_code
            ,CASE
                WHEN j.state IN ('posted','expired','cancelled') THEN j.state
                WHEN j.state = 'osted' THEN 'posted'
                WHEN j.state = 'canceled' THEN 'cancelled'
                ELSE j.state
            END AS state
            ,j.price
            ,CAST(strftime(j.posted_at, '%Y-%m-%d') AS DATE) as posted_at_date
            ,j.posted_at as posted_at_timestamp
            ,CAST(strftime(j.updated_at, '%Y-%m-%d') AS DATE) as updated_at_date
            ,j.updated_at as updated_at_timestamp 
                              
            -- next_updated_at is used to calculate the number of days a job posting has been in each state
            ,LEAD(j.updated_at) OVER (PARTITION BY j.id ORDER BY j.updated_at) AS next_updated_at
        FROM 
            jobs j
    )               
    
    SELECT 
        job_id
        ,company_id
        ,zip_code
        ,state
        ,price
        ,posted_at_date
        ,posted_at_timestamp
        ,updated_at_date
        ,updated_at_timestamp
        ,next_updated_at
        -- days_in_state: get the difference in days between updated_at_date and next_updated_at
        ,CASE
            WHEN next_updated_at IS NOT NULL THEN DATEDIFF('day', updated_at_date, next_updated_at)
            ELSE DATEDIFF('day', updated_at_date, today())
        END AS days_in_state
    FROM 
        next_updated_at
    """).df()

# fill null numerical values with -1 and cast as integer to prevent pandas changing type to float, this ensures no type mismatch when joining tables
fact_job_state[['job_id', 'company_id', 'zip_code', 'days_in_state']] = fact_job_state[['job_id', 'company_id', 'zip_code', 'days_in_state']].fillna(-1)
fact_job_state[['job_id', 'company_id','zip_code', 'days_in_state']] = fact_job_state[['job_id', 'company_id', 'zip_code', 'days_in_state']].astype('Int64') 

fact_job_state = duckdb.query("SELECT * FROM fact_job_state ORDER BY job_id, updated_at_timestamp").df()

display(fact_job_state)

# fact_job_state.to_csv('../power_bi_datasets/fact_job_state.csv', index=False)

,job_id,company_id,zip_code,state,price,posted_at_date,posted_at_timestamp,updated_at_date,updated_at_timestamp,next_updated_at,days_in_state
0,1,4,10506,posted,119.34,2022-01-29,2022-01-29 11:57:00,2022-01-29,2022-01-29 11:57:00,2022-04-13 11:43:00,74
1,1,4,10506,expired,119.34,2022-01-29,2022-01-29 11:57:00,2022-04-13,2022-04-13 11:43:00,NaT,1331
2,2,5,10506,posted,197.89,2020-10-29,2020-10-29 04:51:00,2020-10-29,2020-10-29 04:51:00,2021-01-20 04:54:00,83
3,2,5,10506,cancelled,197.89,2020-10-29,2020-10-29 04:51:00,2021-01-20,2021-01-20 04:54:00,NaT,1779
4,3,8,35786,posted,335.85,2021-05-19,2021-05-19 15:46:00,2021-05-19,2021-05-19 15:46:00,2021-07-21 15:29:00,63
...,...,...,...,...,...,...,...,...,...,...,...
171,98,7,25089,posted,96.56,2021-11-01,2021-11-01 23:38:00,2021-11-01,2021-11-01 23:38:00,2022-01-22 23:15:00,82
172,98,7,25089,expired,96.56,2021-11-01,2021-11-01 23:38:00,2022-01-22,2022-01-22 23:15:00,NaT,1412
173,99,2,25089,posted,164.13,2022-09-15,2022-09-15 09:28:00,2022-09-15,2022-09-15 09:28:00,2022-12-11 09:36:00,87
174,99,2,25089,expired,164.13,2022-09-15,2022-09-15 09:28:00,2022-12-11,2022-12-11 09:36:00,NaT,1089


In [12]:
dim_location = duckdb.query("SELECT zip_code, location FROM locations").df()

display(dim_location)

# dim_location.to_csv('../power_bi_datasets/dim_location.csv', index=False)

,zip_code,location
0,10506,King's Landing
1,80976,Dorne
2,78956,Braavos
3,67305,Harrenhal
4,25089,Winterfell
5,48732,Karhold
6,35786,Tarth
7,93612,Dragonstone


In [23]:
dim_company = duckdb.query("""
    SELECT 
        "Company ID" AS company_id
        ,"Company Name" AS company_name
        ,"Establishment Date" AS establishment_date
        ,"Number of Employees" AS num_employees
    FROM 
        companies
""").df()

display(dim_company)

# dim_company.to_csv('../power_bi_datasets/dim_company.csv', index=False)

,company_id,company_name,establishment_date,num_employees
0,0,Acme Inc.,2019-12-31,404
1,1,Best Corp.,2021-03-21,229
2,2,Bright Future Enterprises,2020-11-14,222
3,3,Delta Inc.,2020-11-06,662
4,4,Echo Enterprises,2022-05-05,255
5,5,Fast Track Inc.,2020-03-08,374
6,6,Global Enterprises,2022-02-03,896
7,7,High Hopes Inc.,2020-03-17,812
8,8,Infinite Solutions,2021-03-12,436
9,9,Jumpstart Corp.,2020-05-08,872


In [ ]:
# Create a date spine for time series analysis

dim_date = duckdb.query("""

    WITH date_range AS (
        SELECT
            MIN(updated_at_date) AS start_date,
            MAX(updated_at_date) AS end_date
        FROM fact_job_state         
    ),
    date_spine AS (
        SELECT
            CAST(unnest(generate_series(
                start_date,
                end_date,
                INTERVAL 1 DAY
            )) AS DATE) AS updated_at_date
        FROM date_range                  
    )
                        
    SELECT DISTINCT
        updated_at_date AS date
        ,date_part('year', updated_at_date) AS year
        ,date_part('month', updated_at_date) AS month
        ,date_part('day', updated_at_date) AS day
        ,date_part('quarter', updated_at_date) AS quarter
        ,strftime(updated_at_date, '%b') AS month_name
        ,date_part('week', updated_at_date) AS week_number
        ,strftime(updated_at_date, '%a') AS day_of_week
        ,CONCAT(date_part('year', updated_at_date), '-', strftime(updated_at_date, '%b')) AS year_month
    FROM 
        date_spine                 
""").df()

display(dim_date)

# dim_date.to_csv('../power_bi_datasets/dim_date.csv', index=False)

,date,year,month,day,quarter,month_name,week_number,day_of_week,year_month
0,2020-10-29,2020,10,29,4,Oct,44,Thu,2020-Oct
1,2020-10-30,2020,10,30,4,Oct,44,Fri,2020-Oct
2,2020-10-31,2020,10,31,4,Oct,44,Sat,2020-Oct
3,2020-11-01,2020,11,1,4,Nov,44,Sun,2020-Nov
4,2020-11-02,2020,11,2,4,Nov,45,Mon,2020-Nov
...,...,...,...,...,...,...,...,...,...
788,2022-12-26,2022,12,26,4,Dec,52,Mon,2022-Dec
789,2022-12-27,2022,12,27,4,Dec,52,Tue,2022-Dec
790,2022-12-28,2022,12,28,4,Dec,52,Wed,2022-Dec
791,2022-12-29,2022,12,29,4,Dec,52,Thu,2022-Dec


## Q1
 - Which location currently has the most jobs in either the active or expired state?

In [15]:
df_q1 = duckdb.query("""
    SELECT
        location
        ,state
        ,COUNT(DISTINCT job_id) as num_jobs
    FROM 
        dim_location l
    LEFT JOIN
        fact_job_state j
        on l.zip_code = j.zip_code
    WHERE state IN ('posted', 'expired')
    GROUP BY location, state
    ORDER BY num_jobs desc
""").df()

display(df_q1)

,location,state,num_jobs
0,King's Landing,posted,23
1,Dorne,posted,22
2,Tarth,posted,22
3,Winterfell,posted,17
4,Braavos,posted,16
5,Tarth,expired,10
6,Winterfell,expired,8
7,King's Landing,expired,6
8,Dorne,expired,6
9,Braavos,expired,4


## Q2
 - In which month were the most jobs cancelled?

In [16]:
df_q2 = duckdb.query("""
    SELECT
        d.year_month
        ,COUNT(DISTINCT j.job_id) as num_jobs
    FROM 
        fact_job_state j
    LEFT JOIN
        dim_date d
        ON j.updated_at_date = d.date
    WHERE state = 'cancelled'
    GROUP BY d.year_month
    ORDER BY count(1) desc
""").df()

display(df_q2)

,year_month,num_jobs
0,2021-Jun,4
1,2022-Aug,3
2,2022-Apr,3
3,2021-May,2
4,2022-Nov,2
5,2022-Jun,2
6,2022-Sep,2
7,2021-Mar,2
8,2021-Jan,2
9,2022-Oct,2


## Q3
 - Which company has the highest ratio of posted jobs to employee count?

In [17]:
df_q3 = duckdb.query("""
    SELECT
        c.company_name
        ,CONCAT(COUNT(j.job_id), ':', c.num_employees) AS job_employee_ratio
        ,(COUNT(j.job_id) / c.num_employees) * 100 AS job_employee_ratio_percent
    FROM 
        fact_job_state j
    LEFT JOIN
        dim_company c
        ON j.company_id = c.company_id
    WHERE state = 'posted'
    GROUP BY c.company_name, c.num_employees
    ORDER BY job_employee_ratio_percent desc
""").df()

display(df_q3)

,company_name,job_employee_ratio,job_employee_ratio_percent
0,Echo Enterprises,13:255,5.098039
1,Best Corp.,10:229,4.366812
2,Bright Future Enterprises,8:222,3.603604
3,Infinite Solutions,12:436,2.752294
4,Fast Track Inc.,10:374,2.673797
5,Acme Inc.,9:404,2.227723
6,Delta Inc.,13:662,1.963746
7,Global Enterprises,16:896,1.785714
8,High Hopes Inc.,10:812,1.231527


## Q4
 - Develop the SQL to define a dimensional model schema for this data.
 - Model created in Fact & Dimension Tables sections above.

## Q5
 - Write a SQL query that returns a list of jobs for each company, ordered and enumerated within each group by the `posted_at` date.

In [18]:
df_q5 = duckdb.query("""
    SELECT
        j.job_id
        ,c.company_name
        ,ROW_NUMBER() OVER (PARTITION BY j.company_id ORDER BY j.posted_at_date) AS company_job_id
        ,j.posted_at_date
        ,j.price
        ,l.location
    FROM 
        fact_job_state j
    LEFT JOIN
        dim_company c
        ON j.company_id = c.company_id
    LEFT JOIN
        dim_location l
        ON j.zip_code = l.zip_code
    WHERE state = 'posted'
    ORDER BY j.company_id, j.posted_at_date asc
""").df()

display(df_q5)

,job_id,company_name,company_job_id,posted_at_date,price,location
0,74,Acme Inc.,1,2021-06-04,464.94,King's Landing
1,51,Acme Inc.,2,2021-07-29,213.84,Winterfell
2,23,Acme Inc.,3,2021-08-12,326.99,Dorne
3,94,Acme Inc.,4,2021-09-11,152.60,King's Landing
4,12,Acme Inc.,5,2021-10-28,106.58,Braavos
...,...,...,...,...,...,...
96,18,Infinite Solutions,8,2022-03-07,78.87,Tarth
97,92,Infinite Solutions,9,2022-03-25,408.29,Winterfell
98,5,Infinite Solutions,10,2022-04-07,158.09,Braavos
99,66,Infinite Solutions,11,2022-06-12,105.04,King's Landing
